The main solving big ridge regression problems

In [1]:
workspace()

In [2]:
using Plots

In [3]:
using StatsBase
using JLD  
include("../src/BigRidge.jl")

plot_outputs_Plots (generic function with 1 method)

In [4]:
#Basic parameters
## Basic parameters
maxiter=10^8;
max_time = 10;
max_epocs = 100;
printiters = true;
exacterror =true; 
tol = 10.0^(-5.0);
restol = 10.0^(-5.0);
skip_error_calculation = 25;   # number of iterations where error is not calculated (to save time!) ceil((m)^(1/2))

In [5]:
dataset = "a9a"
prob = defineProb(dataset);

../data/a9a.jld


In [6]:
options = MyOptions(0,"uni",tol,restol,maxiter,skip_error_calculation,max_time,printiters,exacterror) 
OUTPUTS = [];

In [146]:
Pkg.add("Hadamard")
using Hadamard 

m = 64;
A = rand(m,m); b = A'*rand(m); A = A'*A;  # positive definite


# Solving in one hit
H = hadamard(m);
HA =H*A
Hb =H*b;
xs2 = HA\Hb;
xs1 = A\b;
xs1-xs2; # ok

# Solving in one hit with Newton
x = rand(m);
idx = sample(1:m,m,replace=false);
sa = hada(A,idx);  # S * A
sb = hada(b,idx);  # S * b 
sas = hada(sa',idx); 
 vect = sa*x-sb;
 ST = hada(eye(m),idx);
 y = sas\vect;
 Sy = ST'*y; 
 x[:] = x[:] -Sy;
 println(norm(x-xs2)/norm(xs2))


INFO: Nothing to be done


1.568041059865695e-11


In [13]:
n = 12;
x = rand(n)

idx = sample(1:n,4,replace=false);
n  = size(x,1)
p  = size(x,2)
s = length(idx);

m = Int(floor(n/2));

   # idx_new = idx.*(idx.<=m)+(idx-m).*(idx.>m);

idx_new = idx - m*(idx .> m);
idx 


4-element Array{Int64,1}:
 10
  9
  7
  3

In [14]:
idx_new

4-element Array{Int64,1}:
 4
 3
 1
 3

In [7]:
# Testing Hadamard Sketch
method_name = "Hadamard";# ROS
options.sketchsize =prob.n; # Int(floor(log2(prob.n)));
options.maxiter=100;
options.skip_error_calculation =1;
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

flopsperiter: 1936758
Hadamard
-------------------
It   | Error% | Residual |  Time   
-------------------
  1  | 56.51  |  0.01  | 1.5524 
  2  | 56.51  |  0.01  | 1.6527 
  3  | 56.51  |  0.01  | 1.7602 
  4  | 56.51  |  0.01  | 1.8631 
  5  | 56.51  |  0.01  | 1.9660 
  6  | 56.51  |  0.01  | 2.0724 
  7  | 56.51  |  0.01  | 2.1829 
  8  | 56.51  |  0.01  | 2.2919 
  9  | 56.51  |  0.01  | 2.4042 
 10  | 56.51  |  0.01  | 2.5140 
 11  | 56.51  |  0.01  | 2.6186 
 12  | 56.51  |  0.01  | 2.7175 
 13  | 56.51  |  0.01  | 2.8190 
 14  | 56.51  |  0.01  | 2.9200 
 15  | 56.51  |  0.01  | 3.0255 
 16  | 56.51  |  0.01  | 3.1324 
 17  | 56.51  |  0.01  | 3.2363 
 18  | 56.51  |  0.01  | 3.3442 
 19  | 56.51  |  0.01  | 3.4481 
 20  | 56.51  |  0.01  | 3.5771 
 21  | 56.51  |  0.01  | 3.7958 
 22  | 56.51  |  0.01  | 3.9061 
 23  | 56.51  |  0.01  | 4.0139 
 24  | 56.51  |  0.01  | 4.1223 
 25  | 56.51  |  0.01  | 4.2228 
 26  | 56.51  |  0.01  | 4.3245 
 27  | 56.51  |  0.01  | 4.4259 
 2

In [93]:
# Testing CG 
options.max_time = 2; options.restol = 10.0^(-8); 
method_name = "CG";# gradient descent
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CG
-------------------
It   | Error% | Residual |  Time   
-------------------
abs error: 1.434392412706409
  1  | 96.60  |  29.70  | 0.1243 
abs error: 1.3855765470454868
  2  | 87.44  |  7.62  | 0.1268 
abs error: 1.2541697814483534
  3  | 79.30  |  4.40  | 0.1277 
abs error: 1.1374867345424353
  4  | 74.71  |  2.09  | 0.1286 
abs error: 1.0715711079996741
  5  | 71.27  |  1.68  | 0.1293 
abs error: 1.0223519585221452
  6  | 67.91  |  0.98  | 0.1301 
abs error: 0.9740288288584328
  7  | 65.10  |  0.72  | 0.1309 
abs error: 0.9337198437920716
  8  | 63.25  |  0.57  | 0.1317 
abs error: 0.907246435024017
  9  | 61.39  |  0.38  | 0.1401 
abs error: 0.8805295143472361
 10  | 59.93  |  0.34  | 0.1410 
abs error: 0.8596842125555337
 11  | 58.59  |  0.44  | 0.1418 
abs error: 0.8404447972971131
 12  | 58.51  |  0.27  | 0.1426 
abs error: 0.8392294992058815
 13  | 56.83  |  0.20  | 0.1433 
abs error: 0.8152069386282013
 14  | 54.78  |  0.20  | 0.1441 
abs error: 0.7857572313810305
 15  | 51.

In [73]:
# Testing grad descent
options.max_time = 2; options.restol = 10.0^(-8); options.maxiter=1000;
method_name = "grad";# gradient descent
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

grad
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 73.17  |  1.45  | 0.0249 
 50  | 67.96  |  1.54  | 0.0291 
 75  | 65.17  |  0.44  | 0.0313 
100  | 63.12  |  0.71  | 0.0331 
125  | 61.61  |  0.24  | 0.0347 
150  | 60.29  |  0.43  | 0.0364 
175  | 59.19  |  0.16  | 0.0381 
200  | 58.15  |  0.31  | 0.0398 
225  | 57.25  |  0.12  | 0.0621 
250  | 56.36  |  0.24  | 0.0724 
275  | 55.56  |  0.10  | 0.0738 
300  | 54.77  |  0.20  | 0.0765 
325  | 54.05  |  0.08  | 0.0780 
350  | 53.33  |  0.18  | 0.0794 
375  | 52.67  |  0.08  | 0.0825 
400  | 52.00  |  0.16  | 0.0840 
425  | 51.39  |  0.07  | 0.0854 
450  | 50.76  |  0.15  | 0.0868 
475  | 50.19  |  0.06  | 0.0881 
500  | 49.61  |  0.14  | 0.0900 
525  | 49.07  |  0.06  | 0.0913 
550  | 48.52  |  0.13  | 0.0950 
575  | 48.01  |  0.06  | 0.1022 
600  | 47.49  |  0.12  | 0.1085 
625  | 47.01  |  0.05  | 0.1097 
650  | 46.51  |  0.11  | 0.1189 
675  | 46.06  |  0.05  | 0.1200 
700  | 45.59  |  0.11  | 0.1

In [81]:
# Testing CD 
options.max_time = 10; options.restol = 10.0^(-8); options.maxiter=10000;
method_name = "CD";# gradient descent
options.sketchsize = 11#floor((prob.n)^(1/2));
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CD
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 125.40  |  3.06  | 0.0010 
 50  | 122.14  |  1.19  | 0.0019 
 75  | 117.73  |  0.41  | 0.0028 
100  | 113.81  |  0.22  | 0.0036 
125  | 111.34  |  0.30  | 0.0045 
150  | 109.36  |  0.20  | 0.0053 
175  | 105.00  |  0.18  | 0.0061 
200  | 101.98  |  0.23  | 0.0070 
225  | 100.53  |  0.16  | 0.0079 
250  | 99.47  |  0.15  | 0.0087 
275  | 98.22  |  0.23  | 0.0095 
300  | 96.61  |  0.14  | 0.0104 
325  | 95.79  |  0.14  | 0.0113 
350  | 94.98  |  0.11  | 0.0121 
375  | 94.65  |  0.13  | 0.0129 
400  | 94.95  |  0.08  | 0.0137 
425  | 94.64  |  0.09  | 0.0146 
450  | 94.68  |  0.12  | 0.0154 
475  | 94.73  |  0.10  | 0.0163 
500  | 99.00  |  0.07  | 0.0171 
525  | 100.87  |  0.03  | 0.0179 
550  | 100.83  |  0.02  | 0.0186 
575  | 100.73  |  0.02  | 0.0194 
600  | 100.72  |  0.02  | 0.0213 
625  | 100.79  |  0.01  | 0.0221 
650  | 100.78  |  0.01  | 0.0229 
675  | 100.73  |  0.01  | 0.0237 
700  | 100.69 

In [ ]:
savename = replace(prob.name, r"[\/]", "-");
save("../data/$(savename)-OUTPUTS.jld", "OUTPUTS", OUTPUTS)

In [ ]:
#using GR
using Plots
pgfplots()
plot_outputs_Plots(OUTPUTS,savename)

In [ ]:
# Loading saved problem data
probname = "w8a"#"gisette_scale-ridge";  
test = load("../data/$(probname)-OUTPUTS.jld")
OUTPUTS = test["OUTPUTS"];
# mask = [1,3,5]
# OUTPUTS[mask]